In [1]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
mark = 'tuwen'
mark = 'video'
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/{mark}/8'
# 8
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/{mark}/19'
# 16
info_dir = f'/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/{mark}/11'
info_dir = '/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1'


# info_dir = f'/search/odin/yuwenmengke/pikachu/projects/feed/rank/src/v25_tw_mktest_models_0430/tuwen_0430_tw_withScoreemb_withUseractemb_withDevemb_withDistrib_withKw_noSrch/infos'

day = 20200816

l = []
#info_path = f'{info_dir}/{hour}/valid.csv'
for info_path in glob.glob(f'{info_dir}/{day}*/valid.csv'):
  print(info_path)
  df_ = pd.read_csv(info_path)
  if 'product_data' in df_.columns:
    df_ = df_[df_.product_data=='sgsapp']
  l.append(df_)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})

df = pd.concat(l)

df['dur'] = df.duration
# df['mark'] = 'tuwen'
# info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# info_path = f'{info_dir}/{hour}/valid.csv'
# df2 = pd.read_csv(info_path)
# df2['dur'] = df.duration
# df2['mark'] = 'video'
# df = pd.concat([df, df2])

df.head()

/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081600/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081601/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081602/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081603/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081604/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081605/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081606/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081607/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081608/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081609/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020081610/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/inf

,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,video_time,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur
0,0,0,0,8,0,1,2799558533530813253,0,20814k0449DGan,61,...,61,1,1.0,0,sgsapp,0.045218,0.305020,0.108817,2020081600,61
1,0,0,0,8,0,13,-259214877476560038,3039,20509d0K3MGO99,0,...,90,0,1.0,0,sgsapp,0.016908,0.156587,0.092079,2020081600,0
2,0,0,0,10,0,5,-3612077494153184804,3016,20814k0400VqNo,0,...,156,0,1.0,0,sgsapp,0.014870,0.121849,0.122118,2020081600,0
3,0,0,0,11,0,3,-8134014223018076846,3601,20602m0K0IA1cy,0,...,434,0,1.0,0,sgsapp,0.009589,0.039941,0.517863,2020081600,0
4,0,0,0,5,0,4,-3093577783242455775,7155,20813l044uUg25,-1,...,72,1,1.0,0,sgsapp,0.000877,0.033430,0.023632,2020081600,-1


In [4]:
mark = 'tuwen'
mark = 'video'
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/{mark}/8'
# 8
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/{mark}/19'
# 16
info_dir = f'/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/{mark}/11'
info_dir = '/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1'


# info_dir = f'/search/odin/yuwenmengke/pikachu/projects/feed/rank/src/v25_tw_mktest_models_0430/tuwen_0430_tw_withScoreemb_withUseractemb_withDevemb_withDistrib_withKw_noSrch/infos'

day = 20200716

l = []
#info_path = f'{info_dir}/{hour}/valid.csv'
for info_path in glob.glob(f'{info_dir}/{day}*/valid.csv'):
  print(info_path)
  df_ = pd.read_csv(info_path)
  if 'product_data' in df_.columns:
    df_ = df_[df_.product_data=='sgsapp']
  l.append(df_)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})

df2 = pd.concat(l)

df2['dur'] = df2.duration
# df['mark'] = 'tuwen'
# info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# info_path = f'{info_dir}/{hour}/valid.csv'
# df2 = pd.read_csv(info_path)
# df2['dur'] = df.duration
# df2['mark'] = 'video'
# df = pd.concat([df, df2])
df2.head()

/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071600/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071601/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071602/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071603/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071604/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071605/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071606/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071607/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071608/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071609/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020071610/valid.csv
/search/odin/publicData/CloudS/mkyuwen/rank_online/inf

,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,video_time,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur
0,0,0,0,5,0,11,-1360035706470587875,3028,207099040wYkcF,0,...,81,0,1.0,0,sgsapp,0.002251,0.030298,0.109152,2020071600,0
1,0,0,0,4,0,8,6795342586564692637,3039,20506g0K22qnsE,0,...,91,0,1.0,0,sgsapp,0.009542,0.088861,0.116448,2020071600,0
2,0,0,0,8,0,1,-7132969837828665386,1445,20714b0K3j3Ko4,2,...,32,1,1.0,0,sgsapp,0.052644,0.506943,0.052637,2020071600,2
3,0,0,0,8,0,17,8473210671918860540,3116,20604k0K1i4BJX,0,...,128,0,1.0,0,sgsapp,0.000270,0.006682,0.087592,2020071600,0
4,0,0,0,5,0,2,7876095729357541071,3116,20625i042S0EIG,0,...,32,0,1.0,0,sgsapp,0.007130,0.119920,0.044018,2020071600,0


In [13]:
df

,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,video_time,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur
0,0,0,0,8,0,1,2799558533530813253,0,20814k0449DGan,61,...,61,1,1.0,0,sgsapp,0.045218,0.305020,0.108817,2020081600,61
1,0,0,0,8,0,13,-259214877476560038,3039,20509d0K3MGO99,0,...,90,0,1.0,0,sgsapp,0.016908,0.156587,0.092079,2020081600,0
2,0,0,0,10,0,5,-3612077494153184804,3016,20814k0400VqNo,0,...,156,0,1.0,0,sgsapp,0.014870,0.121849,0.122118,2020081600,0
3,0,0,0,11,0,3,-8134014223018076846,3601,20602m0K0IA1cy,0,...,434,0,1.0,0,sgsapp,0.009589,0.039941,0.517863,2020081600,0
4,0,0,0,5,0,4,-3093577783242455775,7155,20813l044uUg25,-1,...,72,1,1.0,0,sgsapp,0.000877,0.033430,0.023632,2020081600,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892948,0,0,0,10,0,33,-3388587795049433187,3016,2081500K1EVUpK,0,...,110,0,1.0,0,sgsapp,0.008210,0.087314,0.097062,2020081623,0
1892999,0,0,0,13,0,2,-9090847693463699714,3039,20816b0K100jJS,0,...,87,0,1.0,0,sgsapp,0.022950,0.158593,0.139136,2020081623,0
1893050,0,0,0,11,0,27,8922356423131777764,5253,open_doc_prod37934015,0,...,87,0,1.0,0,sgsapp,0.009498,0.093336,0.105598,2020081623,0
1893101,0,0,0,15,0,11,-5564950100949845525,3057,20815j041BKnx0,0,...,67,0,1.0,0,sgsapp,0.005778,0.065748,0.099511,2020081623,0


In [15]:
df2

,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,video_time,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur
0,0,0,0,5,0,11,-1360035706470587875,3028,207099040wYkcF,0,...,81,0,1.0,0,sgsapp,0.002251,0.030298,0.109152,2020071600,0
1,0,0,0,4,0,8,6795342586564692637,3039,20506g0K22qnsE,0,...,91,0,1.0,0,sgsapp,0.009542,0.088861,0.116448,2020071600,0
2,0,0,0,8,0,1,-7132969837828665386,1445,20714b0K3j3Ko4,2,...,32,1,1.0,0,sgsapp,0.052644,0.506943,0.052637,2020071600,2
3,0,0,0,8,0,17,8473210671918860540,3116,20604k0K1i4BJX,0,...,128,0,1.0,0,sgsapp,0.000270,0.006682,0.087592,2020071600,0
4,0,0,0,5,0,2,7876095729357541071,3116,20625i042S0EIG,0,...,32,0,1.0,0,sgsapp,0.007130,0.119920,0.044018,2020071600,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919224,0,0,0,16,0,17,8900917772933674010,5211,20712j0F2gDzDp,0,...,188,0,1.0,0,sgsapp,0.029605,0.141699,0.246759,2020071623,0
1919275,0,0,0,15,0,8,-7762272504662329199,3039,2042450K3IoQHg,411,...,563,1,1.0,0,sgsapp,0.028648,0.165918,0.175637,2020071623,411
1919326,0,0,0,6,0,2,-3730274528534204960,3028,20715j0K1oKPme,0,...,350,0,1.0,0,sgsapp,0.045375,0.162828,0.350828,2020071623,0
1919377,0,0,0,18,0,3,6845408684504380210,0,20712e0K3qgHH7,0,...,461,0,1.0,0,sgsapp,0.058856,0.155323,0.555326,2020071623,0


In [5]:
df.user_active

0          2
1          2
2          0
3          2
4         -1
          ..
1892948    2
1892999    2
1893050    2
1893101    2
1893152    0
Name: user_active, Length: 15468406, dtype: int64

In [6]:
len(df2)

15205084

In [7]:
len(df)

15468406

In [8]:
df.docid

0                 20814k0449DGan
1                 20509d0K3MGO99
2                 20814k0400VqNo
3                 20602m0K0IA1cy
4                 20813l044uUg25
                   ...          
1892948           2081500K1EVUpK
1892999           20816b0K100jJS
1893050    open_doc_prod37934015
1893101           20815j041BKnx0
1893152           208158041JuPwC
Name: docid, Length: 15468406, dtype: object

In [9]:
df2.docid

0          207099040wYkcF
1          20506g0K22qnsE
2          20714b0K3j3Ko4
3          20604k0K1i4BJX
4          20625i042S0EIG
                ...      
1919224    20712j0F2gDzDp
1919275    2042450K3IoQHg
1919326    20715j0K1oKPme
1919377    20712e0K3qgHH7
1919428    20709i041xVM9R
Name: docid, Length: 15205084, dtype: object

In [33]:
d = df.groupby('docid')['docid'].agg({'count'}).reset_index()

In [34]:
d

,docid,count
0,09fc3dd81d66887a8df94bea3bbddf7c,27
1,15f3e2b88b172b37446ebeca5f721666,10
2,197170044C2nuf,18
3,19727m044afrjv,24
4,1980220443tQfd,28
...,...,...
291678,sv_210171735,3
291679,sv_210172098,21
291680,sv_210172282,2
291681,sv_210172834,9


In [35]:
d2 = df2.groupby('docid')['docid'].agg({'count'}).reset_index()

In [36]:
d2

,docid,count
0,09fc3dd81d66887a8df94bea3bbddf7c,15
1,15f3e2b88b172b37446ebeca5f721666,12
2,197170044C2nuf,33
3,19727m044afrjv,25
4,1980220443tQfd,16
...,...,...
312281,sv_204780811,2
312282,sv_204781229,1
312283,sv_204782721,2
312284,sv_204783098,2


In [37]:
old_dids = set(d2.docid)

In [38]:
new_dids = set(d.docid)

In [40]:
len(old_dids - new_dids) / len(old_dids)

0.8866647880468544

In [45]:
d2['count'].describe()

count    312286.000000
mean         48.689611
std         625.090890
min           1.000000
25%           2.000000
50%           5.000000
75%          15.000000
max      128248.000000
Name: count, dtype: float64

In [48]:
len(d2[d2['count'] < 15]) / len(d2)

0.7391205497524705

In [53]:
len(set(d2[d2['count'] < 15].docid) - new_dids) / len(set(d2[d2['count'] < 15].docid))

0.9606918034633497

In [54]:
len(set(d2[d2['count'] < 10].docid) - new_dids) / len(set(d2[d2['count'] < 10].docid))

0.9697328512847888

In [55]:
len(set(d2[d2['count'] < 5].docid) - new_dids) / len(set(d2[d2['count'] < 5].docid))

0.9824016359596516

In [56]:
len(d2[d2['count'] < 5]) / len(d2)

0.4885617670981088

In [57]:
df2 = df2.merge(d2, on='docid', how='left')

In [58]:
len(df2[df2['count'] < 5]) / len(df2)

0.019443365127085124

In [62]:
len(df2[df2['count'] < 500]) / len(df2)

0.41910929265500935

In [63]:
len(set(d2[d2['count'] < 500].docid) - new_dids) / len(set(d2[d2['count'] < 500].docid))

0.8929395896557986

In [60]:
df2

,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur,count
0,0,0,0,5,0,11,-1360035706470587875,3028,207099040wYkcF,0,...,0,1.0,0,sgsapp,0.002251,0.030298,0.109152,2020071600,0,9
1,0,0,0,4,0,8,6795342586564692637,3039,20506g0K22qnsE,0,...,0,1.0,0,sgsapp,0.009542,0.088861,0.116448,2020071600,0,11857
2,0,0,0,8,0,1,-7132969837828665386,1445,20714b0K3j3Ko4,2,...,1,1.0,0,sgsapp,0.052644,0.506943,0.052637,2020071600,2,6166
3,0,0,0,8,0,17,8473210671918860540,3116,20604k0K1i4BJX,0,...,0,1.0,0,sgsapp,0.000270,0.006682,0.087592,2020071600,0,117
4,0,0,0,5,0,2,7876095729357541071,3116,20625i042S0EIG,0,...,0,1.0,0,sgsapp,0.007130,0.119920,0.044018,2020071600,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15205079,0,0,0,16,0,17,8900917772933674010,5211,20712j0F2gDzDp,0,...,0,1.0,0,sgsapp,0.029605,0.141699,0.246759,2020071623,0,12
15205080,0,0,0,15,0,8,-7762272504662329199,3039,2042450K3IoQHg,411,...,1,1.0,0,sgsapp,0.028648,0.165918,0.175637,2020071623,411,39
15205081,0,0,0,6,0,2,-3730274528534204960,3028,20715j0K1oKPme,0,...,0,1.0,0,sgsapp,0.045375,0.162828,0.350828,2020071623,0,8691
15205082,0,0,0,18,0,3,6845408684504380210,0,20712e0K3qgHH7,0,...,0,1.0,0,sgsapp,0.058856,0.155323,0.555326,2020071623,0,23552


In [28]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
d = df.groupby('docid').agg({'docid': 'count'})
d

In [ ]:
dids = set(d.docid)

In [17]:
dids2 = set(d2.docid)

In [20]:
dids2 - dids

{800,
 821,
 834,
 849,
 850,
 922,
 937,
 949,
 958,
 964,
 975,
 987,
 999,
 1016,
 1022,
 1025,
 1030,
 1039,
 1040,
 1046,
 1048,
 1066,
 1068,
 1094,
 1098,
 1110,
 1116,
 1132,
 1140,
 1142,
 1145,
 1157,
 1158,
 1163,
 1166,
 1168,
 1176,
 1186,
 1195,
 1196,
 1224,
 1230,
 1231,
 1233,
 1234,
 1236,
 1244,
 1247,
 1256,
 1260,
 1266,
 1282,
 1286,
 1287,
 1291,
 1292,
 1293,
 1296,
 1301,
 1326,
 1337,
 1338,
 1342,
 1354,
 1356,
 1359,
 1361,
 1365,
 1371,
 1376,
 1380,
 1385,
 1386,
 1394,
 1395,
 1401,
 1416,
 1438,
 1442,
 1447,
 1460,
 1464,
 1469,
 1476,
 1477,
 1480,
 1482,
 1491,
 1495,
 1501,
 1512,
 1516,
 1517,
 1520,
 1521,
 1536,
 1541,
 1549,
 1551,
 1560,
 1566,
 1575,
 1577,
 1579,
 1581,
 1582,
 1583,
 1598,
 1611,
 1612,
 1615,
 1621,
 1624,
 1627,
 1635,
 1639,
 1643,
 1645,
 1653,
 1654,
 1655,
 1658,
 1662,
 1663,
 1664,
 1675,
 1676,
 1678,
 1681,
 1682,
 1683,
 1684,
 1686,
 1690,
 1693,
 1699,
 1702,
 1706,
 1708,
 1711,
 1712,
 1717,
 1721,
 1731,
 1738

In [18]:
d2[d2.docid.isin(dids)]

,docid
docid,
09fc3dd81d66887a8df94bea3bbddf7c,15
15f3e2b88b172b37446ebeca5f721666,12
197170044C2nuf,33
19727m044afrjv,25
1980220443tQfd,16
...,...
sv_204780811,2
sv_204781229,1
sv_204782721,2


In [ ]:
px.pie(df, 'user_active')

In [ ]:
px.pie(df[df.duration!=0], 'user_active')

In [ ]:
df[df.duration!=0].groupby('user_active').agg({'docid': 'count'})

In [ ]:
d = df[df.duration!=0].groupby('mid').first()

In [ ]:
d.groupby('user_active').agg({'docid': 'count'})

In [ ]:
# 全量
mark = 'tuwen'
mark = 'video'
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/{mark}/8'
# 8
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/{mark}/19'
# 16
info_dir = f'/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/{mark}/11'
info_dir = '/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1'

# info_dir = f'/search/odin/yuwenmengke/pikachu/projects/feed/rank/src/v25_tw_mktest_models_0430/tuwen_0430_tw_withScoreemb_withUseractemb_withDevemb_withDistrib_withKw_noSrch/infos'

hour = 2020080222
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df['mark'] = mark
# info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# info_path = f'{info_dir}/{hour}/valid.csv'
# df2 = pd.read_csv(info_path)
# df2['dur'] = df.duration
# df2['mark'] = 'video'
# df = pd.concat([df, df2])
if 'product_data' in df.columns:
  df = df[df.product_data=='sgsapp']
df.head()

In [ ]:
px.pie(df, 'user_active')

In [ ]:
px.pie(df[df.duration!=0], 'user_active')

In [ ]:
df[df.duration!=0].groupby('user_active').agg({'docid': 'count'})

In [ ]:
d = df[df.duration!=0].groupby('mid').first()

In [ ]:
d.groupby('user_active').agg({'docid': 'count'})

In [ ]:
df.hour

In [ ]:
df['click'] = (df.dur != 0).astype(int)

In [ ]:
kv = KV()
titles = kv.get_titles(list(set(df.docid.values)))
df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

In [ ]:
df['mark'] = df.title.apply(lambda x: '苟晶' in x)

In [ ]:
df['click'] = (df.duration != 0).astype(int)

In [ ]:
len(df[df.mark==True])

In [ ]:
len(df[(df.mark==True)&(df.duration!=0)])

In [ ]:
df_doc = df.groupby('docid', as_index=False)["click"].agg({"shows" : "count", "clicks": sum})

In [ ]:
df_doc['ctr'] = df_doc.clicks / df_doc.shows

In [ ]:
df_doc

In [ ]:
df['dur2'] = df.dur.apply(lambda x: x if x > 0 else 0)
df['click2'] = df.dur.apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df_doc2 = df.groupby('docid', as_index=False)["dur2"].agg({"tdur" : sum})
df_doc3 = df.groupby('docid', as_index=False)["click2"].agg({"clicks2" : sum})

In [ ]:
df_doc2

In [ ]:
df_doc3

In [ ]:
df_doc = df_doc.merge(df_doc2, on='docid', how='left')
df_doc = df_doc.merge(df_doc3, on='docid', how='left')

In [ ]:
df_doc

In [ ]:
df_doc['mdur'] = df_doc.tdur / df_doc.clicks2
df_doc = df_doc.fillna(0)
df_doc.mdur = df_doc.mdur.astype(int)
df_doc

In [ ]:
df = df.merge(df_doc, on='docid', how='left')

In [ ]:
df

In [ ]:
df_ = df.docid.value_counts().reset_index().head(20)
df_['title'] = df_['index'].apply(lambda x: titles.get(x, ''))
df_

In [ ]:
def top_recall(df, abid):
  df_ = df[df.abtestid==abid]
  x = df_.docid.value_counts().head(20).reset_index()
  x = x.rename(columns={'index': 'docid', 'docid': 'count'})
  df_doc = df_.groupby('docid', as_index=False)["click"].agg({"shows" : "count", "clicks": sum})
  df_doc['ctr'] = df_doc.clicks / df_doc.shows
  x = x.merge(df_doc, on='docid', how='left')
  x['title'] = x['docid'].apply(lambda x: titles.get(x, ''))
  print(sum(x.clicks))
  print(sum(x.clicks) / sum(x.shows))
  return x

In [ ]:
top_recall(df, 5)

In [ ]:
top_recall(df, 16)

In [ ]:
top_recall(df, 17)

In [ ]:
x = df.docid.value_counts().head(20).reset_index()

In [ ]:
x = x.rename(columns={'index': 'docid', 'docid': 'count'})

In [ ]:
x = x.merge(df_doc, on='docid', how='left')
x['title'] = x['docid'].apply(lambda x: titles.get(x, ''))
x

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option("display.precision", 8)

In [ ]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [ ]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [ ]:
mean_duration = df[df.dur > 0].dur.mean()

In [ ]:
mean_duration

In [ ]:
len(df[df.dur != 0]) / len(df)

In [ ]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [ ]:
import math
df.pred = df.pred.apply(lambda x: math.sqrt(x))

In [ ]:
cold_mids = set(df[df.cold==True].mid.values)

In [ ]:
len(set(df.mid.values))

In [ ]:
len(cold_mids)

In [ ]:
df[df.docid=='20717c0K2Ouv1h'].dur

In [ ]:
df.abtestid

In [ ]:
def debug(abid, mid=None, cold=False):
  while True:
    df_ = df[df.abtestid==abid]
    if mid is None:
      if not cold:
        mid = df_.mid.sample(1).values[0]
      else:
        mid = df_[df_.mid.isin(cold_mids)].mid.sample(1).values[0]
    cols = ['position', 'title', 'dur', 'ori_lr_score', 'pred', 'pred_click', 'pred_dur', 'shows', 'clicks', 'ctr', 'mdur', 'docid']
  #   if 'hour' in df.columns:
  #     cols += ['hour']
    d = df_[df_.mid==mid]
    print('mid:', mid, 'cold:', d.mid.values[0] in cold_mids, 'user_active:', d.user_active.values[0])
    start_time = d.impression_time.min()
    d['time'] = d.impression_time.apply(lambda x: int((x - start_time) / 60))
    cols += ['time']
    d = d.sort_values(['impression_time', 'position'])[cols].reset_index(drop=True)
#     d = d.sort_values(['impression_time', 'pred'], ascending=[True, False])[cols].reset_index(drop=True)
#     return d.sort_values(['ori_lr_score'], ascending=False)[cols].reset_index(drop=True)
    if d.dur.max() > 0:
      break
    else:
      mid = None
  return d

In [ ]:
# mkywuen
debug(5,"c4421593426396019|bbd6f5")

In [ ]:
debug(5, '21d9869114031054017')

In [ ]:
debug(5)

In [ ]:
debug(8)

In [ ]:
len(set(df.impression.values)) 

In [ ]:
len(set(df.mid.values))

In [ ]:
len(set(df.impression.values))  / len(set(df.mid.values))

In [ ]:
len(set(df[df.dur!=0].mid.values))

In [ ]:
len(set(df[df.dur!=0].mid.values)) / len(set(df.mid.values))